# Getting Started with simple-hierarchy

You can also view this notebook in [google colab](https://colab.research.google.com/drive/1wT63yQ4K-XcZRg5Oy-NCt8b4QBDoVE8i?usp=sharing).

Below are two examples of how to use this library.

Please read the [documentation](https://simplehierarchy.readthedocs.io/en/latest/) for more information or check the [PyPI](https://pypi.org/project/simple-hierarchy/) page. 

## Download  simple-hierarchy

In [1]:
!pip install simple-hierarchy

You should consider upgrading via the 'c:\users\rajiv sarvepalli\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


## Example 1

### Creating a Hierarchy 
Below is an image illustrating the class hierarchy. The letter represent the class names and the numbers are the number of classes for that specific class. 

In [2]:
# Create a hierarchy
hierarchy = {
    ("A", 2) : [("B", 5), ("C", 7)],
    ("H", 2) : [("A", 2), ("K", 7), ("L", 10)]
}

### Model Creation and Exploration
We are going to create a model where the parent outputs are fowarded from the last layers to the second to last child layer. The second example will cover how to forward from a specific layer.

In [3]:
from simple_hierarchy.hierarchal_model import HierarchalModel
import torch
import torch.nn as nn

model_b = nn.ModuleList([nn.Linear(10, 10) for i in range(4)])
model = HierarchalModel(
    model=model_b, k=2, hierarchy=hierarchy, size=(10, 10, 10)
)

In [4]:
# the model's tree
model.tree

H 2 [A 2 [B 5 [], C 7 []], K 7 [], L 10 []]

In [5]:
# base layers
model.base_model

Sequential(
  (0): Linear(in_features=10, out_features=10, bias=True)
  (1): Linear(in_features=10, out_features=10, bias=True)
)

In [6]:
# the layers that are distinct per class
# the additional layers are to link together differing output sizes to the provided layers (two additional layers per class)
# in a later version these may be customizable to the layers you want (should you want to proivde distinct aspects of non-linear layers; currently requires conversion into linear layers) 
model.last_layers

ModuleDict(
  (('H', 2)): Sequential(
    (0): Linear(in_features=10, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
  (('A', 2)): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=2, bias=True)
  )
  (('B', 5)): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out_features=10, bias=True)
    (3): Linear(in_features=10, out_features=5, bias=True)
  )
  (('C', 7)): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): Linear(in_features=10, out_features=10, bias=True)
    (2): Linear(in_features=10, out

### Training the model

Training the model is just like training any other PyTorch model, and this model should be able to do everything that every other PyTorch model including things TensorBoard plotting.

## Example 2
The same hierarchy but with parents output being forwarded from specific index.

In [7]:
import torchvision
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

class DemoModel(nn.Module):
    def __init__(self, base_model, size, model_layers, k, feed_from):
        super(DemoModel, self).__init__()
        # Create a hierarchy
        hierarchy = {
            ("A", 2) : [("B", 5), ("C", 7)],
            ("H", 2) : [("A", 2), ("K", 7), ("L", 10)]
        }
        self.model = HierarchalModel(
                        base_model=base_model,
                        hierarchy=hierarchy,
                        size=size,
                        model=model_layers,
                        k=k,
                        feed_from=feed_from,
                    )
    def forward(self, x):
        return self.model(x)

base_model = torchvision.models.resnext101_32x8d(pretrained = True)

c:\users\rajiv sarvepalli\appdata\local\programs\python\python38\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


In [9]:
# these are the indepdent layers of parent and children 
model_layers = [
    nn.Linear(800, 750),
    nn.Linear(750, 512),
    # the output of this layer is feed forward from parent to child
    nn.Linear(512, 128),
    nn.Linear(128, 64),
]

# 1000 is the output size of our base model (the resnext101_32x8d)
# 800 is the input size of our additional indepdent layers (called model_layers)
# 64 is the output size of our additional indepdent layers (called model_layers)
# 128 is the output size of second to last additional indepdent layer to feed forward from parent to child (with concatenation)
size = (1000,800,64,128)
# all 4 layers are distinct for each grouping of classes of model_layers
k = 4
# we want to feed from the second to last layer (from parent to child (with concatenation))
feed_from = 1
model = DemoModel(base_model, size, model_layers, k, feed_from)
model = model.to(device)

In [11]:
out = torch.rand((8, 3,512,512))
pred = model(out)
for p in pred:
    print(p.shape)

torch.Size([8, 2])
torch.Size([8, 2])
torch.Size([8, 5])
torch.Size([8, 7])
torch.Size([8, 7])
torch.Size([8, 10])


In [13]:
model.model.last_layers

ModuleDict(
  (('H', 2)): Sequential(
    (0): Linear(in_features=1000, out_features=800, bias=True)
    (1): Linear(in_features=800, out_features=750, bias=True)
    (2): Linear(in_features=750, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=128, bias=True)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): Linear(in_features=64, out_features=2, bias=True)
  )
  (('A', 2)): Sequential(
    (0): Linear(in_features=1128, out_features=800, bias=True)
    (1): Linear(in_features=800, out_features=750, bias=True)
    (2): Linear(in_features=750, out_features=512, bias=True)
    (3): Linear(in_features=512, out_features=128, bias=True)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): Linear(in_features=64, out_features=2, bias=True)
  )
  (('B', 5)): Sequential(
    (0): Linear(in_features=1128, out_features=800, bias=True)
    (1): Linear(in_features=800, out_features=750, bias=True)
    (2): Linear(in_features=750, out_fe

Here we can use a complex class hierarchy with a complex model in just a few lines of code. Additionally, this model includes feed forwarding from the parent class to the child class from a user-specified layer. In this case, the output of the Linear layer(256, 128) is feed-forward from each parent down into the child. The only independent layer (separate for every grouping of classes) is the Linear layer(128, 64) in this case, but there could be any number of layers as long as the parameters of `k` and `feed_from` are altered accordingly.

The training can be done in the same manner as the first example. The model is just like any other PyTorch model.